# Realized Volatility Models in the OpenBB SDK

Within the Technical Analysis module, there are six models for calculating realized volatility.  Let's have a look at each one.

In [1]:
# Import statements

import pandas as pd
from openbb_terminal.sdk import openbb

## End Points

The cones function is for the rolling quantiles, while the other functions return historical calculations over a specific window. The `cones` function applies a `for` loop over many windows, grouping the results into selectable quantiles.

`windows = [3, 10, 30, 60, 90, 120, 150, 180, 210, 240, 300, 360]`

`openbb.ta.cones()`

`openbb.ta.rvol_garman_klass()`

`openbb.ta.rvol_hodges_tompkins()`

`openbb.ta.rvol_parkinson()`

`openbb.ta_rvol_rogers_satchell()`

`openbb.ta.rvol_std()`

`openbb.ta.rvol_yang_zhang()`

## Collect Some Data

One of the primary steps for most workflows is to capture historical price data. The most robust method is to use the `load` function from the `stocks` module. 

In [2]:
# This will obtain daily price data for the complete history of SPY. The start date only needs to be before the first trading day of SPY, no end date is required to obtain the full history.

df = openbb.stocks.load("SPY", start_date = '1990-01-01')

Loading Daily data for SPY with starting period 1993-01-29.

## Function Inputs and Parameters

The primary input for each function is a Pandas DataFrame containing OHLC data. All other parameters have default settings which do not require additional user input. The `is_crypto` boolean provides an easy switch for the number of trading-days-per-year. 252 is the standard for equities and trad-fi, while `is_crypto = True` enforces 365.


In [3]:
openbb.ta.cones?

Signature:     
openbb.ta.cones(
    data: pandas.core.frame.DataFrame,
    lower_q: float = 0.25,
    upper_q: float = 0.75,
    is_crypto: bool = False,
    model: str = 'STD',
) -> pandas.core.frame.DataFrame
Call signature: openbb.ta.cones(*args: Any, **kwargs: Any) -> Any
Type:           cones
String form:    <openbb_terminal.common.technical_analysis.volatility_model.Operation object at 0x2b29c19f0>
File:           ~/GitHub/OpenBBTerminal/openbb_terminal/common/technical_analysis/volatility_model.py
Docstring:     
Returns a DataFrame of realized volatility quantiles.

Parameters
---------
data: pd.DataFrame
    DataFrame of the OHLC data.
lower_q: float (default = 0.25)
    The lower quantile to calculate the realized volatility over time for.
upper_q: float (default = 0.75)
    The upper quantile to calculate the realized volatility over time for.
is_crypto: bool (default = False)
    If true, volatility is calculated for 365 days instead of 252.
model: str (default = "STD")
  

In [4]:
openbb.ta.rvol_parkinson?

Signature:     
openbb.ta.rvol_parkinson(
    data: pandas.core.frame.DataFrame,
    window: int = 30,
    trading_periods: Optional[int] = None,
    is_crypto: bool = False,
    clean=True,
) -> pandas.core.frame.DataFrame
Call signature: openbb.ta.rvol_parkinson(*args: Any, **kwargs: Any) -> Any
Type:           parkinson
String form:    <openbb_terminal.common.technical_analysis.volatility_model.Operation object at 0x10555b970>
File:           ~/GitHub/OpenBBTerminal/openbb_terminal/common/technical_analysis/volatility_model.py
Docstring:     
Parkinson volatility uses the high and low price of the day rather than just close to close prices.
It is useful for capturing large price movements during the day.

Parameters
----------
data : pd.DataFrame
    Dataframe of OHLC prices.
window : int [default: 30]
    Length of window to calculate over.
trading_periods : Optional[int] [default: 252]
    Number of trading periods in a year.
is_crypto : bool [default: False]
    If true, trading_

## Sample Outputs

Let's take a look at what the expected outputs are. The data we have loaded already is the complete daily OHLC history of SPY. 

In [5]:
df.head(1)

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
date,,,,,,,,
1993-01-29,25.140198,25.140198,25.015122,25.12233,25.12233,1003200,0.0,0.0


### Standard Deviation Model

This model requires a minimum window of two trading periods, the default is thirty. The calculations, by default, are annualized over 252 trading-days-per-year. To change this behaviour:

- `is_crypto = True` to annualize over 365 days.
- Enter an integer value for the `trading_periods` argument. 

In [6]:
openbb.ta.rvol_std(df, window = 1)

Error: Window must be at least 2, defaulting to 30.


date
1993-03-15    0.133723
1993-03-16    0.132503
1993-03-17    0.133976
1993-03-18    0.131752
1993-03-19    0.131355
                ...   
2023-05-16    0.121416
2023-05-17    0.125162
2023-05-18    0.127470
2023-05-19    0.127335
2023-05-22    0.127348
Name: Close, Length: 7603, dtype: float64

For each model there is an override for the `is_crypto` boolean that allows any period to be entered. A message will print indicating that 365 has been overridden. 

In [7]:
openbb.ta.rvol_std(df, trading_periods = 2, window = 365, is_crypto = True)

is_crypto is overridden by trading_periods.


date
1994-07-12    0.008657
1994-07-13    0.008643
1994-07-14    0.008677
1994-07-15    0.008643
1994-07-18    0.008638
                ...   
2023-05-16    0.019547
2023-05-17    0.019534
2023-05-18    0.019537
2023-05-19    0.019518
2023-05-22    0.019458
Name: Close, Length: 7268, dtype: float64

In [8]:
openbb.ta.rvol_std(df, window = 2)

date
1993-02-02    0.055813
1993-02-03    0.094294
1993-02-04    0.071176
1993-02-05    0.054662
1993-02-08    0.007795
                ...   
2023-05-16    0.113925
2023-05-17    0.210704
2023-05-18    0.027824
2023-05-19    0.123960
2023-05-22    0.032422
Name: Close, Length: 7631, dtype: float64

Notice the difference  in the output when the `is_crypto` argument is deployed.

In [9]:
openbb.ta.rvol_std(df, is_crypto = True, window = 2)

date
1993-02-02    0.067172
1993-02-03    0.113483
1993-02-04    0.085660
1993-02-05    0.065786
1993-02-08    0.009382
                ...   
2023-05-16    0.137109
2023-05-17    0.253582
2023-05-18    0.033486
2023-05-19    0.149186
2023-05-22    0.039020
Name: Close, Length: 7631, dtype: float64

In [10]:
openbb.ta.rvol_std(df, window = 10, is_crypto = False)

date
1993-02-12    0.090822
1993-02-16    0.157639
1993-02-17    0.156313
1993-02-18    0.140354
1993-02-19    0.139527
                ...   
2023-05-16    0.122614
2023-05-17    0.130112
2023-05-18    0.125179
2023-05-19    0.094886
2023-05-22    0.094662
Name: Close, Length: 7623, dtype: float64

In [11]:
openbb.ta.rvol_std(df, window = 10, is_crypto = True)

date
1993-02-12    0.109304
1993-02-16    0.189718
1993-02-17    0.188123
1993-02-18    0.168916
1993-02-19    0.167921
                ...   
2023-05-16    0.147566
2023-05-17    0.156589
2023-05-18    0.150653
2023-05-19    0.114196
2023-05-22    0.113926
Name: Close, Length: 7623, dtype: float64

The other models operate in exactly the same way. Let's examine the difference of output.

### Parkinson Model

Print the docstrings with: `help(openbb.ta.rvol_parkinson)`, or in a notebook: `openbb.ta.rvol_parkinson?`

In [12]:
openbb.ta.rvol_parkinson(df)

date
1993-03-12    0.100296
1993-03-15    0.100382
1993-03-16    0.099882
1993-03-17    0.099987
1993-03-18    0.098891
                ...   
2023-05-16    0.095272
2023-05-17    0.095731
2023-05-18    0.097234
2023-05-19    0.096887
2023-05-22    0.096403
Length: 7604, dtype: float64

In [13]:
openbb.ta.rvol_parkinson(df, is_crypto = True)

date
1993-03-12    0.120706
1993-03-15    0.120810
1993-03-16    0.120208
1993-03-17    0.120335
1993-03-18    0.119015
                ...   
2023-05-16    0.114659
2023-05-17    0.115213
2023-05-18    0.117021
2023-05-19    0.116604
2023-05-22    0.116021
Length: 7604, dtype: float64

### Hodges-Tompkins Model

Print the docstrings with: `help(openbb.ta.rvol_hodges_tompkins)`, or in a notebook: `openbb.ta.rvol_hodges_tompkins?`

In [14]:
openbb.ta.rvol_hodges_tompkins(df)

date
1993-03-15    0.134252
1993-03-16    0.133028
1993-03-17    0.134506
1993-03-18    0.132273
1993-03-19    0.131875
                ...   
2023-05-16    0.121897
2023-05-17    0.125657
2023-05-18    0.127974
2023-05-19    0.127839
2023-05-22    0.127852
Name: Close, Length: 7603, dtype: float64

In [15]:
openbb.ta.rvol_hodges_tompkins(df, is_crypto = True)

date
1993-03-15    0.161572
1993-03-16    0.160099
1993-03-17    0.161879
1993-03-18    0.159190
1993-03-19    0.158711
                ...   
2023-05-16    0.146703
2023-05-17    0.151228
2023-05-18    0.154017
2023-05-19    0.153854
2023-05-22    0.153870
Name: Close, Length: 7603, dtype: float64

### Garman-Klass Model

Print the docstrings with: `help(openbb.ta.rvol_garman_klass)`, or in a notebook: `openbb.ta.rvol_garman_klass?`


In [16]:
openbb.ta.rvol_garman_klass(df)

date
1993-03-12    0.090442
1993-03-15    0.090035
1993-03-16    0.089996
1993-03-17    0.089878
1993-03-18    0.089616
                ...   
2023-05-16    0.094239
2023-05-17    0.094939
2023-05-18    0.095223
2023-05-19    0.095100
2023-05-22    0.095307
Length: 7604, dtype: float64

In [17]:
openbb.ta.rvol_garman_klass(df, is_crypto = True)

date
1993-03-12    0.108847
1993-03-15    0.108357
1993-03-16    0.108310
1993-03-17    0.108169
1993-03-18    0.107853
                ...   
2023-05-16    0.113416
2023-05-17    0.114259
2023-05-18    0.114601
2023-05-19    0.114452
2023-05-22    0.114702
Length: 7604, dtype: float64

### Rogers-Satchell Model

Print the docstrings with: `help(openbb.ta.rvol_rogers_satchell)`, or in a notebook: `openbb.ta.rvol_rogers_satchell?`

In [18]:
openbb.ta.rvol_rogers_satchell(df)

date
1993-03-12    0.091722
1993-03-15    0.090740
1993-03-16    0.091090
1993-03-17    0.090891
1993-03-18    0.091437
                ...   
2023-05-16    0.095881
2023-05-17    0.096570
2023-05-18    0.096210
2023-05-19    0.096174
2023-05-22    0.096597
Length: 7604, dtype: float64

In [19]:
openbb.ta.rvol_rogers_satchell(df, is_crypto = True)

date
1993-03-12    0.110387
1993-03-15    0.109205
1993-03-16    0.109627
1993-03-17    0.109387
1993-03-18    0.110045
                ...   
2023-05-16    0.115393
2023-05-17    0.116222
2023-05-18    0.115789
2023-05-19    0.115746
2023-05-22    0.116255
Length: 7604, dtype: float64

### Yang Zhang Model

Print the docstrings with: `help(openbb.ta.rvol_yang_zhang)`, or in a notebook: `openbb.ta.rvol_yang_zhang?`

In [20]:
openbb.ta.rvol_yang_zhang(df)

date
1993-03-15    0.111298
1993-03-16    0.111256
1993-03-17    0.111355
1993-03-18    0.111952
1993-03-19    0.107311
                ...   
2023-05-16    0.118857
2023-05-17    0.120784
2023-05-18    0.120858
2023-05-19    0.120798
2023-05-22    0.119659
Length: 7603, dtype: float64

In [21]:
openbb.ta.rvol_yang_zhang(df, is_crypto = True)

date
1993-03-15    0.133947
1993-03-16    0.133896
1993-03-17    0.134016
1993-03-18    0.134734
1993-03-19    0.129149
                ...   
2023-05-16    0.143044
2023-05-17    0.145363
2023-05-18    0.145453
2023-05-19    0.145380
2023-05-22    0.144009
Length: 7603, dtype: float64

### Cones

This function generates the realized volatility quantiles and returns a DataFrame which can be used for plotting the cones. There are two functions, one is for returning the DataFrame only, the other is for returning a chart.

`openbb.ta.cones()`

`openbb.ta.cones_chart()`

For these functions, the lower and upper quantiles have been parameterized. Use the `lower_q` and `upper_q` arguments as a float between 0 and 1, representing the % value. The default model is "STD", `rvol_std`.

In [22]:
openbb.ta.cones(df)

,Realized,Min,Lower 25%,Median,Upper 75%,Max
3,0.067051,0.000020,0.064342,0.112175,0.186071,1.919686
10,0.090822,0.020061,0.088717,0.133426,0.193801,1.130632
30,0.133723,0.035120,0.101007,0.137116,0.199344,0.864874
60,0.120574,0.049786,0.107721,0.141828,0.200107,0.755843
90,0.114354,0.054997,0.110296,0.142329,0.202462,0.646355
120,0.108223,0.062658,0.110734,0.144296,0.206262,0.590355
150,0.100045,0.064545,0.112083,0.150372,0.206060,0.560031
180,0.096711,0.066184,0.111582,0.152916,0.209219,0.522603
210,0.094264,0.067330,0.112947,0.156255,0.212414,0.490693
240,0.091158,0.067542,0.113469,0.155373,0.218571,0.464774


In [23]:
openbb.ta.cones(df, lower_q = 0.05, upper_q = 0.95)

,Realized,Min,Lower 5%,Median,Upper 95%,Max
3,0.067051,0.000020,0.025351,0.112175,0.359045,1.919686
10,0.090822,0.020061,0.056420,0.133426,0.336100,1.130632
30,0.133723,0.035120,0.068505,0.137116,0.318638,0.864874
60,0.120574,0.049786,0.073103,0.141828,0.320132,0.755843
90,0.114354,0.054997,0.075948,0.142329,0.309737,0.646355
120,0.108223,0.062658,0.082614,0.144296,0.316700,0.590355
150,0.100045,0.064545,0.086252,0.150372,0.308093,0.560031
180,0.096711,0.066184,0.085992,0.152916,0.344487,0.522603
210,0.094264,0.067330,0.088853,0.156255,0.346878,0.490693
240,0.091158,0.067542,0.091431,0.155373,0.335409,0.464774


Sanity checks are in place to ensure that lower and upper are always labeled correctly.

In [24]:
openbb.ta.cones(df, lower_q = 0.95)

,Realized,Min,Lower 75%,Median,Upper 95%,Max
3,0.067051,0.000020,0.186071,0.112175,0.359045,1.919686
10,0.090822,0.020061,0.193801,0.133426,0.336100,1.130632
30,0.133723,0.035120,0.199344,0.137116,0.318638,0.864874
60,0.120574,0.049786,0.200107,0.141828,0.320132,0.755843
90,0.114354,0.054997,0.202462,0.142329,0.309737,0.646355
120,0.108223,0.062658,0.206262,0.144296,0.316700,0.590355
150,0.100045,0.064545,0.206060,0.150372,0.308093,0.560031
180,0.096711,0.066184,0.209219,0.152916,0.344487,0.522603
210,0.094264,0.067330,0.212414,0.156255,0.346878,0.490693
240,0.091158,0.067542,0.218571,0.155373,0.335409,0.464774


Use the `model` argument to select one of the other models.

In [25]:
openbb.ta.cones(df, model = 'Yang-Zhang')

,Realized,Min,Lower 25%,Median,Upper 75%,Max
3,0.206475,0.036251,0.157369,0.235232,0.357460,2.901305
10,0.156358,0.041593,0.151526,0.218055,0.315054,1.852409
30,0.209220,0.069355,0.159284,0.216444,0.310035,1.317698
60,0.181869,0.080218,0.166396,0.221705,0.312657,1.135741
90,0.172319,0.091529,0.170240,0.221417,0.316227,0.976418
120,0.161437,0.096184,0.171186,0.224732,0.326131,0.906005
150,0.150722,0.098014,0.174459,0.231825,0.329826,0.847694
180,0.148022,0.101196,0.175054,0.233376,0.329538,0.789502
210,0.145173,0.103581,0.177064,0.242534,0.333237,0.741164
240,0.140835,0.103185,0.178854,0.241279,0.338688,0.702875


In [26]:
openbb.ta.cones(df, model = 'Rogers-Satchell')

,Realized,Min,Lower 25%,Median,Upper 75%,Max
3,0.053685,0.015730,0.075360,0.108269,0.158418,1.103625
10,0.085904,0.031829,0.081241,0.114853,0.162057,0.903857
30,0.091722,0.037450,0.085884,0.119179,0.166372,0.738473
60,0.085163,0.047437,0.089691,0.123566,0.169604,0.628890
90,0.082095,0.051270,0.093332,0.127600,0.173094,0.560099
120,0.076862,0.053294,0.094080,0.129863,0.177516,0.523743
150,0.072831,0.053766,0.095223,0.131771,0.179267,0.487082
180,0.072897,0.056494,0.095474,0.130743,0.181531,0.453949
210,0.072318,0.057753,0.096278,0.131573,0.180664,0.426170
240,0.069716,0.058269,0.097581,0.133427,0.184192,0.403736


In [27]:
openbb.ta.cones(df, model = 'Garman-Klass')

,Realized,Min,Lower 25%,Median,Upper 75%,Max
3,0.049987,0.020195,0.076182,0.109392,0.158780,1.088747
10,0.077443,0.032200,0.081161,0.114087,0.162890,0.866488
30,0.090442,0.036694,0.085880,0.117537,0.165943,0.711636
60,0.083531,0.045660,0.089302,0.122893,0.170142,0.616402
90,0.080789,0.049871,0.092375,0.125330,0.174088,0.542882
120,0.075991,0.052002,0.093272,0.127634,0.178225,0.508204
150,0.072065,0.052709,0.094731,0.130014,0.179766,0.474018
180,0.071635,0.054395,0.094966,0.128983,0.178643,0.441896
210,0.071099,0.056442,0.095941,0.131505,0.180758,0.415168
240,0.068787,0.056761,0.097031,0.132806,0.181215,0.394028


### Chart the Realized Volatility Cones

Apply the same syntax to, `openbb.ta.cones_chart()`, to visualize the output.

In [28]:
openbb.ta.cones_chart(df)

In [29]:
openbb.ta.cones_chart(df, model = 'Garman-Klass')

The `data` input can also be the `load` function from the OpenBB SDK. Use the `symbol` argument to add the name of the security to the title. Notice that the values shift with the amount of historical data being fed to the model.

In [30]:
openbb.ta.cones_chart(
    data = openbb.stocks.load('SPY'),
    symbol = 'SPY',
    model = 'Garman-Klass',
)

Loading Daily data for SPY with starting period 2020-05-18.

In [31]:
openbb.ta.cones_chart(
    data = openbb.stocks.load('VOO'),
    symbol = 'VOO',
    model = 'Garman-Klass',    
)

Loading Daily data for VOO with starting period 2020-05-18.

## Advanced Use

A chart or a table is nice and all, but what are we supposed to do with it? Becoming a Script Kitty can start by simply being curious.  Here's a really basic example of what curiousity can look like:

### How can I compare the difference between all models within one table?

In [32]:
# Get some data - TLT is the iShares 20 Plus Year Treasury Bond ETF. 

data = openbb.stocks.load('TLT', start_date = '2000-01-01')

Loading Daily data for TLT with starting period 2002-07-30.

In [43]:
# Visualize the data, if desired.

openbb.ta.ma_chart(data = data["Close"], window = [50,252], symbol = "iShares 20 Plus Year Treasury Bond ETF")

In [34]:
# Create a Pandas DataFrame and then assign columns to each model.

results = pd.DataFrame()

results['Standard Deviation'] = openbb.ta.rvol_std(data, window = 2)
results['Parkinson'] = openbb.ta.rvol_parkinson(data, window = 2)
results['Hodges-Tompkins'] = openbb.ta.rvol_hodges_tompkins(data, window = 2)
results['Garman-Klass'] = openbb.ta.rvol_garman_klass(data, window = 2)
results['Rogers-Satchell'] = openbb.ta.rvol_rogers_satchell(data, window = 2)
results['Yang-Zhang'] = openbb.ta.rvol_yang_zhang(data, window = 2)

results

,Standard Deviation,Parkinson,Hodges-Tompkins,Garman-Klass,Rogers-Satchell,Yang-Zhang
date,,,,,,
2002-08-01,0.074478,0.083403,0.074507,0.075624,0.070795,0.140886
2002-08-02,0.050613,0.105981,0.050632,0.103655,0.096629,0.144342
2002-08-05,0.064936,0.109263,0.064961,0.114035,0.109442,0.163509
2002-08-06,0.145800,0.072415,0.145856,0.078072,0.079528,0.138412
2002-08-07,0.123229,0.083376,0.123276,0.092822,0.096105,0.152725
...,...,...,...,...,...,...
2023-05-16,0.083098,0.056754,0.083130,0.056731,0.057372,0.203401
2023-05-17,0.000993,0.074319,0.000994,0.073768,0.073247,0.161466
2023-05-18,0.050694,0.062928,0.050714,0.060527,0.057119,0.108147


The "problem" has been solved; achieving a sense of accomplishment is important for maintaining the motivation required to keep driving forward. Breaking large workflows down to a series of smaller items will make them easier to manage, make the undertaking seem less overwhelming, and provide the satisfaction associated with completing a task or chore.

With the question answered, what is the next task? 

### How do I parametize this table so that it can be called as a one-liner function, sporting a variable ticker input?

The first step is to wrap the DataFrame above in a function, with the input parameters of the function corresponding to those belonging to the individual model one-liners.

In [35]:
from typing import Optional

def rvol(
    data: pd.DataFrame, 
    window:int = 2,
    trading_periods: Optional[int] = None,
    is_crypto: bool = False
)-> pd.DataFrame:
    
    results = pd.DataFrame()

    results['Standard Deviation'] = openbb.ta.rvol_std(data, window, trading_periods, is_crypto)
    results['Parkinson'] = openbb.ta.rvol_parkinson(data, window, trading_periods, is_crypto)
    results['Hodges-Tompkins'] = openbb.ta.rvol_hodges_tompkins(data, window, trading_periods, is_crypto)
    results['Garman-Klass'] = openbb.ta.rvol_garman_klass(data, window, trading_periods, is_crypto)
    results['Rogers-Satchell'] = openbb.ta.rvol_rogers_satchell(data, window, trading_periods, is_crypto)
    results['Yang-Zhang'] = openbb.ta.rvol_yang_zhang(data, window, trading_periods, is_crypto)
    
    return results

rvol(data)

,Standard Deviation,Parkinson,Hodges-Tompkins,Garman-Klass,Rogers-Satchell,Yang-Zhang
date,,,,,,
2002-08-01,0.074478,0.083403,0.074507,0.075624,0.070795,0.140886
2002-08-02,0.050613,0.105981,0.050632,0.103655,0.096629,0.144342
2002-08-05,0.064936,0.109263,0.064961,0.114035,0.109442,0.163509
2002-08-06,0.145800,0.072415,0.145856,0.078072,0.079528,0.138412
2002-08-07,0.123229,0.083376,0.123276,0.092822,0.096105,0.152725
...,...,...,...,...,...,...
2023-05-16,0.083098,0.056754,0.083130,0.056731,0.057372,0.203401
2023-05-17,0.000993,0.074319,0.000994,0.073768,0.073247,0.161466
2023-05-18,0.050694,0.062928,0.050714,0.060527,0.057119,0.108147


Then, because the `data` argument will accept a function as an input, the `load` function can be passed through, instead of a Pandas DataFrame. First, setup the parameters as a separate block.

In [45]:
symbol: str = 'SPY'
window: int = 2
trading_periods: int = 252
is_crypto: bool = False

Now make the function to wrap both the `load` and `rvol` together.

In [46]:
def realized_vol(symbol) -> pd.DataFrame:
    
    rvol_df = rvol(openbb.stocks.load(f"{symbol}"), window, trading_periods, is_crypto)
    
    return rvol_df
    
rvol_df = realized_vol(symbol)

rvol_df

Loading Daily data for SPY with starting period 2020-05-18.

,Standard Deviation,Parkinson,Hodges-Tompkins,Garman-Klass,Rogers-Satchell,Yang-Zhang
date,,,,,,
2020-05-20,0.304978,0.110756,0.305785,0.114520,0.121462,0.281731
2020-05-21,0.266851,0.104713,0.267557,0.111753,0.110010,0.268624
2020-05-22,0.099061,0.106203,0.099324,0.114822,0.116273,0.162092
2020-05-26,0.116154,0.161547,0.116462,0.177137,0.197524,0.440696
2020-05-27,0.028337,0.213748,0.028412,0.240633,0.288865,0.553026
...,...,...,...,...,...,...
2023-05-16,0.113925,0.067305,0.114226,0.073176,0.079609,0.123906
2023-05-17,0.210704,0.095065,0.211262,0.097077,0.096350,0.171472
2023-05-18,0.027824,0.117387,0.027898,0.107230,0.098561,0.171242


In [47]:
realized_vol('AAPL')

Loading Daily data for AAPL with starting period 2020-05-18.

,Standard Deviation,Parkinson,Hodges-Tompkins,Garman-Klass,Rogers-Satchell,Yang-Zhang
date,,,,,,
2020-05-20,0.281262,0.133725,0.282007,0.141057,0.159839,0.294854
2020-05-21,0.300210,0.123866,0.301005,0.128683,0.127003,0.266105
2020-05-22,0.156040,0.134516,0.156453,0.137112,0.133793,0.196616
2020-05-26,0.148334,0.182548,0.148727,0.140428,0.109928,0.280800
2020-05-27,0.125097,0.202267,0.125428,0.181842,0.173497,0.330996
...,...,...,...,...,...,...
2023-05-16,0.032570,0.085883,0.032656,0.090960,0.099904,0.146813
2023-05-17,0.040373,0.111616,0.040480,0.125234,0.138693,0.192023
2023-05-18,0.111990,0.142642,0.112287,0.140982,0.141147,0.206233


The possibilities are bound only by the imagination!